In [44]:
os.getcwd()

'C:\\Users\\Leonardo\\OneDrive\\Documents\\TU_Delft\\CodingProjects\\PoliceBrutality\\notebooks'

In [1]:
import requests
import os
import json
import time
import datetime
import dateutil.parser

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
file_name_base = '../data/raw/full_archive_test/' #enter path where you want the file saved to

bearer_token = "AAAAAAAAAAAAAAAAAAAAAI1PNgEAAAAATarF1xcbxxXdqM15JyHfxL8TLqk%3DTbEmtm5q9UZ0rDTUQa0wUerv9w3LHZAX3UM9gn3qDNvFHhTB56"#os.environ.get("BEARER_TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
# query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
query_params = {'query': '(george OR floyd OR police OR brutality OR black OR lives OR matter OR blacklivesmatter OR violence) (#blacklivesmatter OR #blm OR #georgefloyd) lang:en has:geo place_country:US',
#                 'tweet.fields': 'author_id,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld', point_radius:[-73.935242 40.730610 25mi]
                'tweet.fields': 'author_id,conversation_id,created_at,geo,id,in_reply_to_user_id,referenced_tweets',
                'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
                'max_results': '10',
                'expansions': 'geo.place_id',
#                 'user.fields': 'created_at,description,id,location,name,protected,public_metrics,url,username,verified',
                'start_time': '2020-05-26T00:00:00.00Z',
                'end_time': '2020-05-27T00:00:00.00Z'}


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(search_url, headers, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))
    ts = json_response['data'][-1]['created_at']
    stamp = dateutil.parser.parse(ts).strftime('%m%d%H%M%S_%y')
    # save one file for each response
    file_name = file_name_base + stamp + '.json'
    with open(file_name , 'w') as f:
        json.dump(json_response['data'], f)

    print("Collected", len(json_response['data']), "tweets from", json_response['data'][0]['created_at'], "to", ts)
    # update query with "next_token" from the most recent tweet to go to the next search page
    query_params['next_token'] = json_response['meta']['next_token']
    
#     return json_response

# if __name__ == "__main__":
# #     test = main()
#     main()

    
if __name__ == "__main__":
    while(True):
        try:
            main()  
        except:
            time.sleep(10)
            print("program restart")